### Machine Translation with Seq2Seq Model in PyTorch

# Machine Translation
Machine translation (MT) refers to the automatic translation of text from one language to another using computational models. Neural Machine Translation (NMT) systems often use Seq2Seq architectures with encoder-decoder structures and attention mechanisms.

## What is Machine Translation?

Machine Translation (MT) is a subfield of Natural Language Processing (NLP) focused on the automatic translation of text or speech from one language to another. Given a source sentence $ X = (x_1, x_2, \dots, x_n) $ in a source language, the goal is to generate a target sentence $ Y = (y_1, y_2, \dots, y_m) $ in a target language that conveys the same meaning.

In mathematical terms, the task can be modeled as finding the most probable translation $ Y^* $:

$$
Y^* = \underset{Y}{\text{argmax}} \; P(Y|X)
$$

Where:
- $ P(Y|X) $ is the conditional probability of the target sequence given the source sequence.

Seq2Seq models break this task into two steps:
1. **Encoding:** The source sentence $ X $ is encoded into a fixed-length context vector $ C $, which represents the meaning of the source sentence.
2. **Decoding:** The target sentence $ Y $ is generated word-by-word based on the context vector $ C $ and the previously generated words.




In [48]:
!pip install datasets

In [49]:
!pip install torchmetrics

This lab demonstrates how to:
1. Load a translation dataset from Hugging Face's `datasets` library.
2. Create a PyTorch Dataset and DataLoader.
3. Build a Seq2Seq model with an encoder-decoder architecture.
4. Train the model.
5. Evaluate the model using BLEU score.

### 1. Load  Dataset
The **ManyThings English-French dataset** is a popular dataset used for machine translation tasks, particularly for English-to-French translations. It is simple, lightweight, and primarily intended for beginner-level experimentation with machine translation models. Below is an explanation of its key aspects:

**Content:**
   - The dataset consists of parallel sentences, meaning that each English sentence is paired with its corresponding translation in French.
   - The translations are often short, conversational, and simple, making it ideal for introductory machine translation tasks.



In [50]:
from datasets import load_dataset

ds = load_dataset("avitri/eng-fra")

In [51]:
ds

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 135842
    })
})

In [52]:
split_ds=ds["train"].train_test_split(0.1, seed=33)

In [53]:
train_data, test_data= split_ds['train'], split_ds['test']

In [54]:
train_data[0]['text'].split('\t')

['They bought a box of cookies.', 'Ils ont acheté une boîte de biscuits.']

In [55]:
train_tokens_A=[token for sentence_pair in  train_data for token in sentence_pair['text'].split('\t')[0].lower().split()]
train_tokens_B=[token for sentence_pair in  train_data for token in sentence_pair['text'].split('\t')[1].lower().split()]

In [56]:
from collections import Counter

vocab_A=Counter(train_tokens_A)
vocab_B=Counter(train_tokens_B)



In [57]:
w2i_A={k: (i+4) for i, (k,v) in enumerate(vocab_A.items())}
w2i_B={k: (i+4) for i, (k,v) in enumerate(vocab_B.items())}

In [58]:
for i, k in enumerate(['<pad>','<bos>','<eos>','<unk>']):
     w2i_A[k]=i
     w2i_B[k]=i

In [59]:
len(w2i_A),len(w2i_B)

(22603, 37272)

### 2. Dataset and Dataloader Creation

In [60]:
from torch.utils.data import Dataset
class CustomDataset(Dataset):
    def __init__(self, train_data, w2i_A, w2i_B):
        self.train_data = train_data
        self.w2i_A = w2i_A
        self.i2w_A = {v:k for (k,v) in w2i_A.items()}

        self.w2i_B = w2i_B
        self.i2w_B = {v:k for (k,v) in w2i_B.items()}
    def __len__(self):
        return len(self.train_data)

    def __getitem__(self, idx):
        sample = self.train_data[idx]
        sent_A, sent_B= sample['text'].split('\t')


        tokens_A=[w2i_A['<bos>']]
        for token in sent_A.lower().split():
          if token in w2i_A:
            tokens_A.append(w2i_A[token])
          else:
             tokens_A.append(w2i_A['<unk>'])
        tokens_A.append(w2i_A['<eos>'])

        tokens_B=[w2i_B['<bos>']]
        for token in sent_B.lower().split():
          if token in w2i_B:
            tokens_B.append(w2i_B[token])
          else:
             tokens_B.append(w2i_B['<unk>'])
        tokens_B.append(w2i_B['<eos>'])

        return tokens_A, tokens_B, len(tokens_A) , len(tokens_B)

In [61]:
train_dataset=CustomDataset(train_data, w2i_A, w2i_B)
test_dataset=CustomDataset(test_data, w2i_A, w2i_B)

In [62]:
train_dataset[0]

([1, 4, 5, 6, 7, 8, 9, 2], [1, 4, 5, 6, 7, 8, 9, 10, 2], 8, 9)

In [63]:
test_dataset[0]

([1, 25, 35, 11722, 12, 3, 23, 2342, 8, 6837, 2],
 [1, 38, 158, 17229, 3, 17, 68, 6659, 9, 9384, 2],
 11,
 11)

In [64]:
import torch
from torch.nn.utils.rnn import pad_sequence

def my_collate_fn(batch):
    """
    Custom collate function for the DataLoader to handle variable-length sequences.
    Pads input and target sequences to the maximum length in the batch.

    Args:
        batch (list of dicts): Each element contains `source_ids`, `source_mask`, and `target_ids`.

    Returns:
        dict: Padded and batched inputs and targets.
    """


    # Separate source and target sequences
    source_ids = [torch.tensor(item[0]) for item in batch]
    target_ids = [torch.tensor(item[1]) for item in batch]

    source_lentghs=torch.tensor( [item[2] for item in batch])
    target_lentghs = torch.tensor( [item[3] for item in batch])
    # Pad the sequences to the maximum length in the batch
    padded_source_ids = pad_sequence(source_ids, batch_first=True, padding_value=0)
    padded_target_ids = pad_sequence(target_ids, batch_first=True, padding_value=0)


    return padded_source_ids, padded_target_ids , source_lentghs, target_lentghs

In [65]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn= my_collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

### 3. Seq2Seq Model Definition

In [66]:
import torch.nn as nn
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = torch.nn.functional.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights



In [67]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, padding_idx_A, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size, padding_idx=padding_idx_A)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

In [68]:
class Seq2Seq(nn.Module):
    def __init__(self, num_embeddings_A, num_embeddings_B, hidden_size, padding_idx_A, padding_idx_B , dropout_rate=0.1):
        super(Seq2Seq, self).__init__()

        self.encoder= EncoderRNN(num_embeddings_A, hidden_size, padding_idx_A, dropout_rate)
        self.embedding = nn.Embedding(num_embeddings_B, hidden_size, padding_idx=padding_idx_B)

        self.attention = BahdanauAttention(hidden_size)

        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, num_embeddings_B)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, input_tensor, target_tensor):

        encoder_outputs, encoder_hidden=self.encoder(input_tensor)
        batch_size,seq_len = target_tensor.shape

        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(seq_len):

            decoder_input = target_tensor[:, i].unsqueeze(1)
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)


        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

In [69]:
# Initialize encoder and decoder
num_embeddings_A=len(w2i_A)
padding_idx_A=w2i_A['<pad>']


num_embeddings_B=len(w2i_B)
padding_idx_B=w2i_B['<pad>']

hidden_size= 256
dropout_rate=0.1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model= Seq2Seq(num_embeddings_A, num_embeddings_B ,hidden_size,padding_idx_A, padding_idx_B,dropout_rate).to(device)


In [70]:
len(w2i_A),len(w2i_B)

(22603, 37272)

In [71]:
!gdown --id 1p7bsrla6DGgtJNOAuqOmAn6FQ5YgrHDH


/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1p7bsrla6DGgtJNOAuqOmAn6FQ5YgrHDH

but Gdown can't. Please check connections and permissions.


In [72]:
from torchmetrics.text import Perplexity

In [73]:
# Define Loss and Optimizer
optimizer = torch.optim.Adam(model.parameters())
loss_fn= torch.nn.CrossEntropyLoss().to(device)

perp=Perplexity().to(device)

### 4. Training Loop

In [74]:
import tqdm
import numpy as np
n_epochs=10
for epoch in range(n_epochs):
  running_perplexity=[]
  running_loss=[]
  with tqdm.tqdm(train_dataloader, unit='batch') as tepoch:
    for batch in tepoch:
      tepoch.set_description(f'Epoch {epoch}')

      padded_source_ids, padded_target_ids , source_lentghs, target_lentghs = batch
      padded_source_ids, padded_target_ids =padded_source_ids.to(device).long(), padded_target_ids.to(device).long()

      # Zero your gradients for every batch!
      optimizer.zero_grad()
      outputs, attentions =model(padded_source_ids, padded_target_ids)

      batch_size, seq_len, vocab_size = outputs.shape

      outputs = outputs[:,:-1,:].reshape(-1, vocab_size)
      targets = padded_target_ids[:,1:].reshape(-1)

      # Create a mask for non-padding tokens
      mask = targets != 0

      # Apply mask to filter out padding tokens
      filtered_outputs = outputs[mask]
      filtered_targets = targets[mask]

      loss = loss_fn(outputs, targets)

      loss.backward()
      optimizer.step()
      perplexity = perp(outputs.reshape(batch_size, (seq_len-1), vocab_size), targets.reshape(batch_size, (seq_len-1)))

      running_perplexity.append(perplexity.item())
      running_loss.append(loss.item())
      tepoch.set_postfix(loss=np.mean(running_loss), perplexity = np.mean(running_perplexity) )


Epoch 9: 100%|██████████| 1911/1911 [03:01<00:00, 10.51batch/s, loss=0.331, perplexity=1.4]


In [75]:
torch.save(model.state_dict(), 'pretrained_mt.pth')

In [76]:
model.load_state_dict(torch.load('pretrained_mt.pth'))

<ipython-input-76-30c48d22b986>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('pretrained_mt.pth'))


<All keys matched successfully>

In [77]:
#from nltk.translate.bleu_score import sentence_bleu